In [1]:
# Import BeautifulSoup
from bs4 import BeautifulSoup
import re
import time

In [2]:
# Import Splinter and set the chromedriver path
from splinter import Browser
executable_path = {"executable_path": "chromedriver"}
browser = Browser("chrome", **executable_path, headless=False)

### NASA Mars News

In [3]:
# Visit the following URL
url = "https://mars.nasa.gov/news/"
browser.visit(url)

In [4]:
# Scrape the browser into soup 
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
first_article = soup.find(class_="slide")

In [5]:
title = first_article.find(class_="content_title").text
title

"NASA's Curiosity Keeps Rolling As Team Operates Rover From Home"

In [6]:
paragraph = first_article.find(class_="article_teaser_body").text
paragraph

'The team has learned to meet new challenges as they work remotely on the Mars mission.'

### JPL Mars Space Images - Featured Image

In [7]:
# Use splinter to navigate the site and find the image url for the current Featured Mars Image and 
# assign the url string to a variable called `featured_image_url`.
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

In [8]:
# Make sure to find the image url to the full size `.jpg` image.
find_full_image = browser.find_by_id('full_image')
find_full_image.click()

# Get the more info button and click it
browser.is_element_present_by_text('more info')
find_more_info = browser.links.find_by_partial_text('more info')
find_more_info.click()

# Get the html for the image
image_html = BeautifulSoup(browser.html, 'html.parser')
image_html

<!DOCTYPE html>
<!--[if IE 9]> <html class="no-js ie ie9" lang="en"> <![endif]--><!--[if IE 8]> <html class="no-js ie ie8" lang="en"> <![endif]--><html class="js flexbox canvas canvastext webgl no-touch geolocation postmessage websqldatabase indexeddb hashchange history draganddrop websockets rgba hsla multiplebgs backgroundsize borderimage borderradius boxshadow textshadow opacity cssanimations csscolumns cssgradients cssreflections csstransforms csstransforms3d csstransitions fontface generatedcontent video audio localstorage sessionstorage webworkers applicationcache svg inlinesvg smil svgclippaths -webkit-" style="" xmlns="http://www.w3.org/1999/xhtml"><!-- START HEADER: "DEFAULT" --><!-- Google Tag Manager --><head><script async="" src="https://www.google-analytics.com/analytics.js" type="text/javascript"></script><script src="https://m.addthis.com/live/red_lojson/300lo.json?si=5e9632ee3a32c539&amp;bkl=0&amp;bl=1&amp;pdt=558&amp;sid=5e9632ee3a32c539&amp;pub=&amp;rev=v8.28.3-wp&amp

In [9]:
# get relative image url
relative_url = image_html.select_one('figure.lede a img').get("src")
relative_url         

'/spaceimages/images/largesize/PIA16101_hires.jpg'

In [10]:
# Save a complete url string for the image
# concatenate the domain name to get the full featured_image_url 
featured_image_url = 'https://www.jpl.nasa.gov' + relative_url  
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA16101_hires.jpg'

### Mars Weather

In [11]:
# Visit the Mars Weather twitter account. Scrape the latest Mars weather tweet from the page. 
# Save the tweet text for the weather report as a variable called `mars_weather`.

url = "https://twitter.com/marswxreport?lang=en"
browser.visit(url)
time.sleep(5)

In [12]:
# Scrape the browser into soup 

html = browser.html
soup = BeautifulSoup(html, 'html.parser')

soup

<!DOCTYPE html>
<html dir="ltr" lang="en" style="font-size: 15px;" xmlns="http://www.w3.org/1999/xhtml"><head><meta charset="utf-8"/>
<meta content="width=device-width,initial-scale=1,maximum-scale=1,user-scalable=0,viewport-fit=cover" name="viewport"/>
<link href="//abs.twimg.com" rel="preconnect"/>
<link href="//api.twitter.com" rel="preconnect"/>
<link href="//pbs.twimg.com" rel="preconnect"/>
<link href="//t.co" rel="preconnect"/>
<link href="//video.twimg.com" rel="preconnect"/>
<link href="//abs.twimg.com" rel="dns-prefetch"/>
<link href="//api.twitter.com" rel="dns-prefetch"/>
<link href="//pbs.twimg.com" rel="dns-prefetch"/>
<link href="//t.co" rel="dns-prefetch"/>
<link href="//video.twimg.com" rel="dns-prefetch"/>
<link as="script" crossorigin="anonymous" href="https://abs.twimg.com/responsive-web/web/polyfills.bab5fe74.js" nonce="" rel="preload"/>
<link as="script" crossorigin="anonymous" href="https://abs.twimg.com/responsive-web/web/vendors~main.e0482f54.js" nonce="" rel="

In [23]:
find_tweet = soup.find("div", attrs = {"class":"tweet", "data-name":"mars_weather"} )
find_tweet

In [24]:
# Example: mars_weather = 'Sol 1801 (Aug 30, 2017), Sunny, high -21C/-5F, 
# low -80C/-112F, pressure at 8.82 hPa, daylight 06:09-17:55'

try:
    mars_weather = find_tweet.find("p","tweet-text").get_text()
    mars_weather
except AttributeError:
    pattern = re.compile(r"sol")
    mars_weather = soup.find("span", text=pattern).text
    mars_weather 
mars_weather

'InSight sol 491 (2020-04-14) low -93.2ºC (-135.7ºF) high -3.6ºC (25.5ºF)\nwinds from the SW at 5.3 m/s (11.9 mph) gusting to 16.9 m/s (37.8 mph)\npressure at 6.60 hPa'

### Mars Facts

In [16]:
# Visit the Mars Facts webpage and 
# use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.

# Use Pandas to convert the data to a HTML table string.

import pandas as pd
facts_df = pd.read_html('https://space-facts.com/mars/')[0]
facts_df.head()

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"


In [17]:
facts_df.columns=['category', 'value']
facts_df.head()

,category,value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"


In [18]:
# remove default index and set index on category
facts_df.set_index('category', inplace=True)
facts_df.head()

,value
category,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"


### Mars Hemispheres

In [21]:
# Visit the USGS Astrogeology site to obtain high resolution images for each of Mar's hemispheres.

url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url)
time.sleep(1)

In [22]:
# You will need to click each of the links to the hemispheres in order to find the image url 
# to the full resolution image

hemisphere_list=[]
desc = browser.find_by_css("a.product-item h3")
for i in range(len(desc)):
    
    # Save both the image url string for the full resolution hemisphere image, and the 
    # Hemisphere title containing the hemisphere name. Use a Python dictionary to store 
    # the data using the keys `img_url` and `title`.

    hemisphere = {}
    browser.find_by_css("a.product-item h3")[i].click()
    image = browser.find_link_by_text("Sample").first
    hemisphere["img_url"] = image["href"]
    hemisphere["titles"] = browser.find_by_css("h2.title").text
    
    # Append the dictionary with the image url string and the hemisphere title to a list. 
    # This list will contain one dictionary for each hemisphere.
    
    hemisphere_list.append(hemisphere)
    browser.back()
    
hemisphere_list

[{'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
  'titles': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
  'titles': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
  'titles': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
  'titles': 'Valles Marineris Hemisphere Enhanced'}]